In [ ]:
import numpy as np

基础检查

In [ ]:
from data_drive import DRIVEDataset
from data_drive import get_drive_loaders
import torch
ds = DRIVEDataset(
                  root='/root/autodl-tmp/1KIMI/DRIVE',
                  image_ids=[21, 22, 23],
                  img_size=512,
                  in_channels=1,
                  is_training=True
                  )
print(ds)
item = ds[0]
print(f"Length of returned item: {len(item)}")
print(f"Type of returned item: {type(item)}")

img, mask, _= ds[0]
print(f"Image range: [{img.min():.3f}, {img.max():.3f}]")  # 应为[0,1]或[-1,1]
print(f"Mask unique: {torch.unique(mask)}")               # 应为[0., 1.]
print(f"Mask mean: {mask.mean():.4f}")                    # 应在0.06-0.12之间
train_loader, val_loader, _ = get_drive_loaders()
for imgs, masks, _ in train_loader:
    print(f"Batch shape: {imgs.shape}")  # 应为 [4, 1, 256, 256]（batch=4, channel=1）
    print(f"Masks shape: {masks.shape}") # 应为 [4, 1, 256, 256]
    print(f"Mask sum: {masks.sum()}")    # 应为正数（有血管像素）
    break
# 1. 创建一个包含ID 37的数据集实例
ds_37 = DRIVEDataset(
    root='/root/autodl-tmp/1KIMI/DRIVE',
    image_ids=[37],  # 只关注ID 37
    img_size=512,
    in_channels=1,
    is_training=False # 验证集通常不进行增强，设为False更符合验证逻辑，虽然路径模式主要取决于ID
)

# 2. 获取并打印路径
# ds_37.img_dir 是根据ID 37初始化后的目录对象
# ds_37.img_pattern 是文件名格式化字符串
img_path_37 = ds_37.img_dir / ds_37.img_pattern.format(37)

print(f"第37号图像的路径是: {img_path_37}")


Cubical Ripser测试代码

In [ ]:
import numpy as np
# 假设用A
import cripser

# 生成测试图像（必须是 uint8 类型！）
img = np.random.randint(0, 256, (128, 128), dtype=np.uint8)
# 计算0维持续图
result = cripser.computePH(img, maxdim=0)
print(result)  # 期望输出: [(birth, death), ...]
print(f"结果形状: {result.shape}")

CubicalRipser输出格式确认：Birth/Death是滤过值（阈值），不是坐标

In [ ]:
import numpy as np
import cripser

# 构造真正的双连通分量图像
test_img = np.ones((100, 100), dtype=np.uint8) * 255  # 背景白色（255）

# 上半部分方块：值为 50（较暗，先出现）
test_img[20:30, 20:80] = 50

# 下半部分方块：值为 150（较亮，后出现）
test_img[60:70, 20:80] = 150

# 计算持续图
result = cripser.computePH(test_img, maxdim=0)
dgm0 = result[result[:, 0] == 0][:, [1, 2]]  # 提取 birth, death

print("持续图 (birth, death):")
print(dgm0)
print(f"\n共发现 {len(dgm0)} 个连通分量")

# 解析
for i, (birth, death) in enumerate(dgm0):
    if np.isinf(death):
        print(f"分量 {i+1}: birth={birth}, death=inf (背景)")
    else:
        life = death - birth
        print(f"分量 {i+1}: birth={birth}, death={death}, 寿命={life}")

# 预期结果：
# 分量 1: birth=50, death=150, 寿命=100 (上半方块，与下半合并时死亡)
# 分量 2: birth=150, death=inf (下半方块，吞并上半后成为主分量)
# 或者反过来，取决于哪个先出现

金标Betti计算

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import label
import numpy as np
from data_drive import DRIVEDataset

dataset = DRIVEDataset(
    root='./DRIVE',
    image_ids=[21, 22, 23],
    img_size=512,
    in_channels=1
)
image, vessel, roi = dataset[0]
vessel_np = vessel.squeeze().numpy()
roi_np = roi.squeeze().numpy()
vessel_roi = (vessel_np * (roi_np > 0)).astype(np.uint8)

# 计算连通分量并标记
labeled_array, num_features = label(vessel_roi, structure=np.array([[0,1,0],[1,1,1],[0,1,0]]))

print(f"总组件数: {num_features}")

# 统计各组件面积（像素数）
component_sizes = np.bincount(labeled_array.ravel())[1:]  # 跳过背景0
print(f"组件面积统计: 最小={component_sizes.min()}, 最大={component_sizes.max()}, 平均={component_sizes.mean():.1f}")

# 可视化最大的10个组件
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(min(10, num_features)):
    mask = (labeled_array == (i+1)).astype(np.uint8)
    axes[i].imshow(mask, cmap='gray')
    axes[i].set_title(f'ID:{i+1}, Size:{component_sizes[i]}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# 查看面积分布直方图
plt.figure(figsize=(10, 4))
plt.hist(component_sizes, bins=50, range=(0, 200))
plt.xlabel('Component Size (pixels)')
plt.ylabel('Count')
plt.title(f'Distribution of {num_features} Components')
plt.axvline(x=10, color='r', linestyle='--', label='10 pixels threshold')
plt.legend()
plt.show()

In [ ]:
import yaml
from data_combined import get_combined_loaders

cfg = {
    'data': {
        'use_kaggle_combined': True,
        'image_size': 512
    },
    'training': {
        'batch_size': 4
    }
}

print("开始测试Kaggle模式（首次会自动下载，耗时5-10分钟）...")
train, val, test = get_combined_loaders(cfg)

print(f"✅ 训练集批次: {len(train)}")
print(f"✅ 验证集批次: {len(val)}")

batch = next(iter(train))
img, vessel, roi = batch
print(f"✅ 图像形状: {img.shape}")
print(f"✅ Mask形状: {vessel.shape}")
print(f"✅ ROI全1验证: {roi.unique()}")  # 应该全是1.0

print("\n🎉 Kaggle模式验证通过！明早可直接训练。")